In [1]:
import sys; sys.path.append("../")

import pandas as pd

from clean.brusly_pd_cprr_2020 import clean as clean_cprr

In [2]:
df = clean_cprr()
df

,date_of_notice,date_of_incident,officer_name,supervisor_name,violation,action_taken,start_date,return_date
0,5/3/19,5/3/19,Brett Portier,Asst. Chief Tom Southon,Failure to Report in a Timely Manner,First Offense Counseling,NaN,NaN
1,5/5/20,4/29/20,David Johnson,Lt. Jason Fontenot,Conduct Unbecoming of an Officer,Suspended Without Pay,5/13/20,5/29/20


In [3]:
df.columns

Index(['date_of_notice', 'date_of_incident', 'officer_name', 'supervisor_name',
       'violation', 'action_taken', 'start_date', 'return_date'],
      dtype='object')